<a href="https://colab.research.google.com/github/JensBlack/IEECR_Hackathon24/blob/main/IEECR_Hackathon_24_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IEECR HACKATHON 2024

## Titanic Dataset
sourced from seaborn example datasets

Definition	Key

survival |	Survival 	|0 = No, 1 = Yes

pclass |Ticket class |	1 = 1st, 2 = 2nd, 3 = 3rd

sex |	Sex

Age |	Age in years

sibsp |	# of siblings / spouses aboard the Titanic

parch |	# of parents / children aboard the Titanic

ticket |	Ticket number

fare |	Passenger fare

cabin |	Cabin number

embarked |	Port of Embarkation |	C = Cherbourg, Q = Queenstown, S = Southampton

# Prepping everything (just run once)

In [1]:
#@title Install all necessary libraries (run cell)

! pip install numpy pandas matplotlib scipy scikit-learn seaborn

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
data = sns.load_dataset("titanic")

cleaned_data = data.copy(deep = True)
#data cleaning

## add missing values in age with mean age

age_idx = np.argwhere(cleaned_data["age"].isna()).flatten()
cleaned_data["age"].iloc[age_idx] = cleaned_data.age.mean()

## the deck column has almost no values, so let's drop it

cleaned_data = cleaned_data.drop(columns = ["deck"])

## remove people that are unknown in their embarkment
emb_idx = np.argwhere(cleaned_data["embarked"].isna()).flatten()
cleaned_data = cleaned_data.drop(index= emb_idx, axis=0)

targets = cleaned_data.survived
features = cleaned_data.drop(columns = "survived")

<ipython-input-3-f4555b2917b7>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data["age"].iloc[age_idx] = cleaned_data.age.mean()


In [4]:
features.head()

,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,3,male,22.0,1,0,7.2500,S,Third,man,True,Southampton,no,False
1,1,female,38.0,1,0,71.2833,C,First,woman,False,Cherbourg,yes,False
2,3,female,26.0,0,0,7.9250,S,Third,woman,False,Southampton,yes,True
3,1,female,35.0,1,0,53.1000,S,First,woman,False,Southampton,yes,False
4,3,male,35.0,0,0,8.0500,S,Third,man,True,Southampton,no,True


In [8]:
from sklearn.preprocessing import LabelEncoder

#Convert features to be usable

## binary encode sex
adpt_features = features.copy(deep = True)

#drop columns that are useless or cheating
adpt_features.drop(columns = ["adult_male", "embark_town", "who", "class", "alive"], inplace = True)

# female = 1, male = 0

n_sex = np.zeros_like(adpt_features.sex)
n_sex[np.argwhere(adpt_features.sex == "female").flatten()] = 1
adpt_features.sex = n_sex

# alone and alive are both binary

adpt_features.alone = adpt_features.alone.astype(int)

# # yes = 1, no = 0
# n_alive = np.zeros_like(adpt_features.alive)
# n_alive[np.argwhere(adpt_features.alive == "yes").flatten()] = 1

adpt_features.alive = n_alive

## embark needs to be reencoded to numbers, same with class
l_encoder = LabelEncoder()
n_embarked = l_encoder.fit_transform(adpt_features.embarked)
print(l_encoder.classes_)
adpt_features.embarked = n_embarked


adpt_features.head()

['C' 'Q' 'S']


<ipython-input-8-efdb92427bff>:25: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  adpt_features.alive = n_alive


,pclass,sex,age,sibsp,parch,fare,embarked,alone
0,3,0,22.0,1,0,7.2500,2,0
1,1,1,38.0,1,0,71.2833,0,0
2,3,1,26.0,0,0,7.9250,2,1
3,1,1,35.0,1,0,53.1000,2,0
4,3,0,35.0,0,0,8.0500,2,1


In [9]:
# split the data
train_data, test_data, train_targets, test_targets = train_test_split(adpt_features, targets
                                                    , test_size=0.3
                                                    , random_state=42)

train_data.shape, test_data.shape

((622, 8), (267, 8))

In [10]:
from sklearn.ensemble import RandomForestClassifier


X_train, X_val, y_train, y_val = train_test_split(train_data, train_targets
                                                    , test_size=0.2
                                                    , random_state=42)

print(X_train.shape, X_val.shape)

# train the model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# predict the validation set
y_pred = clf.predict(X_val)

# calculate the accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Validation accuracy: {:.2f}%".format(accuracy * 100))


(497, 8) (125, 8)
Validation accuracy: 80.00%


## Testing on the test set

In [11]:
y_pred = clf.predict(test_data)

# calculate the accuracy
accuracy = accuracy_score(test_targets, y_pred)

print("Test accuracy: {:.2f}%".format(accuracy * 100))


Test accuracy: 76.40%
